In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey
from haversine import haversine

In [2]:
# load standard prepared data fail
fire_data = pd.read_csv("Fire.csv")
fire_data.head()

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783,5000,1.01,1.23
1,1,17962.0,10/30/2019,03:42 AM,75229.0,7,A,13.0,DWELLING,1.0,...,525470.0,1.87,NaN,NaN,10728 LATHROP DR,32.894248,-96.857108,10000,0.97,1.26
2,2,17963.0,10/30/2019,10:47 AM,75241.0,5,B,3.0,DWELLING,1.0,...,286000.0,6.54,NaN,NaN,6205 S R L THORNTON SERV,32.694543,-96.824020,20000,1.88,2.06
3,3,17964.0,10/30/2019,05:32 PM,75243.0,2,B,10.0,DWELLING,1.0,...,114710.0,9.47,NaN,NaN,12044 WILLOWDELL DR,32.913584,-96.764435,12000,0.27,1.43
4,4,17965.0,10/30/2019,11:03 AM,75212.0,9,B,6.0,DWELLING,1.0,...,29000.0,65.48,NaN,NaN,2212 LAPSLEY ST,32.769986,-96.908816,55000,0.46,2.37


In [3]:
#Search for 2 nearest fire stations, find latitude and longitude
places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# needed to clear output below
from IPython.display import clear_output

dist0 = []
dist1 = []
length = len(fire_data)
status = ""

for index, row in fire_data.iterrows():
    
    # store current lat long
    lat = row['Lat']
    lng = row['Lng']
    latlng = str(lat) + ',' + str(lng)
    
    params = {
        "location": latlng,
        "key": gkey,
        "type": "fire_station",
        'rankby': 'distance'
    }
    
    try:
        geo_data = requests.get(places_url, params=params).json()
        
        # record lat & long for first two results
        lat0 = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng0 = geo_data["results"][0]["geometry"]["location"]["lng"]
        lat1 = geo_data["results"][1]["geometry"]["location"]["lat"]
        lng1 = geo_data["results"][1]["geometry"]["location"]["lng"]
        if (geo_data['status'] == 'INVALID_REQUEST'):
            status = "Invalid Request."
        else:
            status = 'found.'
        
        dist0.append(haversine(lat,lng,lat0,lng0,'mi'))
        dist1.append(haversine(lat,lng,lat1,lng1,'mi'))
        
    except:
        lat0 = None
        lng0 = None
        lat1 = None
        lng1 = None
        status = 'error.'
        dist0.append(None)
        dist1.append(None)
    
    # Show status while running, but without the clutter.
    clear_output(wait=True)
    print(f"{index+1} of {length}: {status}  Dist0: {dist0[index]}, Dist1: {dist1[index]}")
    
    
    

621 of 621: found.  Dist0: 1.1872932191257024, Dist1: 1.7557976327439737


In [9]:
# load new data into dataframe
fire_data['Dist to 1st Station (mi)'] = dist0
fire_data['Dist to 2nd Station (mi)'] = dist1

# round data to 2 decimal points
fire_data['Dist to 1st Station (mi)'] = fire_data['Dist to 1st Station (mi)'].round(2)
fire_data['Dist to 2nd Station (mi)'] = fire_data['Dist to 2nd Station (mi)'].round(2)
fire_data.head(10)

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS,Dist to 1st Station (mi),Dist to 2nd Station (mi)
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783,5000,1.01,1.23
1,1,17962.0,10/30/2019,03:42 AM,75229.0,7,A,13.0,DWELLING,1.0,...,525470.0,1.87,NaN,NaN,10728 LATHROP DR,32.894248,-96.857108,10000,0.97,1.26
2,2,17963.0,10/30/2019,10:47 AM,75241.0,5,B,3.0,DWELLING,1.0,...,286000.0,6.54,NaN,NaN,6205 S R L THORNTON SERV,32.694543,-96.824020,20000,1.88,2.06
3,3,17964.0,10/30/2019,05:32 PM,75243.0,2,B,10.0,DWELLING,1.0,...,114710.0,9.47,NaN,NaN,12044 WILLOWDELL DR,32.913584,-96.764435,12000,0.27,1.43
4,4,17965.0,10/30/2019,11:03 AM,75212.0,9,B,6.0,DWELLING,1.0,...,29000.0,65.48,NaN,NaN,2212 LAPSLEY ST,32.769986,-96.908816,55000,0.46,2.37
5,5,17966.0,10/31/2019,09:25 PM,75212.0,9,C,6.0,DWELLING,1.0,...,34880.0,5.42,NaN,NaN,4007 ARANSAS ST,32.790290,-96.875248,2000,1.21,1.73
6,6,17967.0,10/31/2019,11:49 AM,75216.0,5,C,4.0,APARTMENTS,2.0,...,2595000.0,1.14,NaN,NaN,3540 WILHURT AVE,32.713743,-96.763392,30000,0.80,2.23
7,7,17968.0,10/31/2019,11:07 AM,75227.0,8,C,7.0,APARTMENTS,2.0,...,248000.0,0.48,NaN,NaN,4243 OSBORN RD,32.787776,-96.694828,1200,0.97,1.36
8,8,17969.0,10/31/2019,08:13 AM,75232.0,5,C,4.0,MOTEL,1.0,...,817800.0,0.85,NaN,NaN,5715 S R L THORNTON FRWY,32.676848,-96.823544,7000,1.03,2.05
9,9,17263.0,11/01/2019,01:53 AM,75228.0,8,C,7.0,APARTMENTS,2.0,...,4689000.0,0.13,NaN,NaN,3550 ST FRANCIS AVE,32.798165,-96.691340,6000,0.66,1.71


In [10]:
# Check data for missing results
fire_data.loc[fire_data['Dist to 2nd Station (mi)'].isna()]

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS,Dist to 1st Station (mi),Dist to 2nd Station (mi)
29,29,17275.0,11/08/2019,06:37 PM,75240.0,2,B,11.0,APARTMENTS,2.0,...,1970000.0,1.50,NO,YES,13740 MONTFORT DR,32.935232,-96.810022,30000,NaN,NaN
32,32,17280.0,11/09/2019,07:50 AM,75229.0,7,C,13.0,DWELLING,1.0,...,193080.0,0.52,NO,YES,3056 WILDFLOWER DR,32.901950,-96.875667,1000,NaN,NaN
40,40,17290.0,11/11/2019,09:20 PM,75287.0,2,B,12.0,APARTMENTS,3.0,...,2980000.0,1.00,YES,YES,3840 FRANKFORD RD,32.995566,-96.848082,30000,NaN,NaN
44,44,17295.0,11/12/2019,11:52 PM,75252.0,2,C,12.0,DWELLING,2.0,...,667407.0,1.48,NO,YES,18206 BRIGHTON GREEN,32.998482,-96.786273,10000,NaN,NaN
45,45,17296.0,11/12/2019,09:58 PM,75216.0,6,C,4.0,DWELLING,1.0,...,100000.0,7.41,NaN,NaN,1427 STELLA AVE,32.727821,-96.803137,8000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,588,18423.0,09/06/2020,11:00 PM,75240.0,2,B,11.0,APARTMENTS,2.0,...,15003700.0,0.01,NO,YES,5854 PRESTON VIEW BLVD,32.934075,-96.807532,800,NaN,NaN
595,595,18475.0,09/24/2020,06:51 PM,75224.0,6,B,4.0,DWELLING,1.0,...,174670.0,1.96,NO,NO,2441 S LLEWELLYN AVE,32.717737,-96.831143,3500,NaN,NaN
598,598,18432.0,09/10/2020,05:23 PM,75243.0,4,C,10.0,DWELLING,1.0,...,137600.0,0.04,NO,YES,8328 CAMPANELLA DR,32.918196,-96.758976,50,NaN,NaN
604,604,18440.0,09/12/2020,01:43 PM,75225.0,4,C,13.0,COMMERCIAL,2.0,...,726499700.0,0.00,YES,YES,8687 N CENTRAL SERV SB,32.868487,-96.773600,300,NaN,NaN


In [12]:
# save back into group's fire csv file
fire_data.to_csv(r'Fire.csv', index = False)

In [6]:
# configuration guide: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, 96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout)

coordinates = fire_data[['Lat','Lng']]

station_dist = fire_data['Dist to 1st Station (mi)']
heat_layer = gmaps.heatmap_layer(coordinates, weights=station_dist, dissipating=True, max_intensity=5, point_radius=20)
fig.add_layer(heat_layer)

fig
# existing fires and distance to closest fire station

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [7]:
gmaps.configure(api_key=gkey)
dallas_coords = (32.7767, 96.7970)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    
}
fig = gmaps.figure(layout=figure_layout)

coordinates = fire_data[['Lat','Lng']]

station_dist = fire_data['Dist to 2nd Station (mi)']
heat_layer = gmaps.heatmap_layer(coordinates, weights=station_dist, dissipating=True, max_intensity=5, point_radius=20)
fig.add_layer(heat_layer)

fig
# existing fires and distance to 2nd-closest fire station

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))